<a href="https://colab.research.google.com/github/DoHyung08/RL/blob/main/0406myEnv/designing_my_environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 강화 학습 환경 복습

In [ ]:
!pip install gymnasium[classic-control]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.4 MB/s eta 0:00:00


### 강화 학습 문제를 직접 풀어낼 정책 정의

강화 학습에서는 어떤 함수를 학습하고자 하는 걸까요? 에이전트 안에는 상태 관측값(입력)을 받고 그것을 앞으로 취해야 할 최적의 행동(출력)에 매핑하는 함수가 있습니다. 예를 들어, 미로 속 에이전트의 현재 상태가 $(2, 3)$ 좌표라면, 에이전트 안의 함수는 이 입력값을 "오른쪽으로 이동"이라는 출력값에 매핑하는 것이 될 수 있습니다. 이 함수를 $\pi$라고 한다면, 아래와 같이 수식으로 쓸 수 있습니다.
$$
\pi((2, 3)) = \text{"오른쪽으로 이동"}
$$
강화 학습 용어로 이 함수를 정책(policy)이라고 부릅니다.

In [ ]:
def policy(state):
    x_pos, y_pos = state
    if x_pos == 2 and y_pos == 3:
        return +1
    else:
        return 0

### 강화 학습이 돌아가는 환경의 코드 복습

1. 인공지능 모델은 환경의 현재 상태(state)를 관찰할 수 있습니다. 미로 찾기 문제에서 환경의 현재 상태란 미로 속 현재 위치를 의미합니다. 예를 들어, 모델이 미로의 $(2, 3)$ 좌표에 있다면, 이 좌표는 현재 상태를 나타냅니다.

2. 인공지능 모델은 관찰된 상태로부터 앞으로 취할 행동(action)을 결정합니다. 양갈래 길 중에서 어디로 갈지 결정하는 것 등이 그 예시가 될 수 있습니다.

3. 환경은 상태를 변경(transition)시키고 그 행동에 대한 보상(reward)을 생성합니다. 인공지능 모델은 그 상태와 보상을 다 받습니다. 미로 찾기 문제에서 환경의 변화란 인공지능 모델의 (앞선 결정에 따른) 미로 속 위치 변화를 의미합니다. 예를 들어, '오른쪽으로 이동' 행동을 취하면, 에이전트의 위치 좌표가 $(2, 3)$에서 $(2, 4)$로 바뀔 수 있습니다. 보상은 출구를 찾았을 때 주어지는 경품이나 막다른 길에 도달했을 때 받는 페널티 등을 생각해 볼 수 있습니다.

4.  이 새로운 정보(환경의 변화와 이에 따른 보상)를 사용하여 인공지능은 그런 행동이 좋아 그걸 반복해야 하는지, 또는 좋지 않아 회피해야 하는지 결정할 수 있습니다. 완료될 때까지 (done) 이 관측-행동-보상 사이클은 계속됩니다.

In [ ]:
import gymnasium as gym

env = gym.make('MountainCar-v0')
state, _ = env.reset()
print("Initial state:", state)

done = False
total_reward = 0
while not done:
    action = policy(state) # Step 1-2: Observes the state and chooses an action
    print("Chose action:", action)
    state, reward, done, _, _ = env.step(action) # Step 3: Environment returns the next state and reward
    total_reward += reward
    print("New state:", state)
    print("Reward:", reward)
    if total_reward < -200:
        break

print("Final state:", state)
print("Total reward:", total_reward)
env.close()


Initial state: [-0.52625793  0.        ]
Chose action: 0
New state: [-0.527238   -0.00098006]
Reward: -1.0
Chose action: 0
New state: [-0.5291908  -0.00195276]
Reward: -1.0
Chose action: 0
New state: [-0.5321016  -0.00291083]
Reward: -1.0
Chose action: 0
New state: [-0.53594863 -0.00384706]
Reward: -1.0
Chose action: 0
New state: [-0.5407031  -0.00475446]
Reward: -1.0
Chose action: 0
New state: [-0.5463293  -0.00562623]
Reward: -1.0
Chose action: 0
New state: [-0.5527852  -0.00645588]
Reward: -1.0
Chose action: 0
New state: [-0.5600225  -0.00723727]
Reward: -1.0
Chose action: 0
New state: [-0.5679871  -0.00796463]
Reward: -1.0
Chose action: 0
New state: [-0.5766198 -0.0086327]
Reward: -1.0
Chose action: 0
New state: [-0.58585656 -0.00923672]
Reward: -1.0
Chose action: 0
New state: [-0.59562904 -0.0097725 ]
Reward: -1.0
Chose action: 0
New state: [-0.6058655  -0.01023646]
Reward: -1.0
Chose action: 0
New state: [-0.6164912  -0.01062571]
Reward: -1.0
Chose action: 0
New state: [-0.627429

# 환경 코드 예시 살펴보기

### 복도를 걸어다니며 배정된 방까지 이동하는 환경
강화 학습 환경을 더 직관적으로 이해하기 위해 직접 환경 코드를 만들어 봅시다. 강화 학습의 환경을 만들기 위해서는 먼저 상태 공간 $S$와 행동 공간 $A$를 정의해야 합니다.

예를 들어, 일자형 복도에서 배정받은 방을 찾아 돌아다니는 환경을 생각해봅시다. 여러분은 왼쪽 또는 오른쪽으로 이동할 수 있습니다. 이 환경에서 상태와 행동은 아래와 같이 표현할 수 있습니다.
\begin{equation}
S = \{(i, j): i, j \in \{\text{Room 101}, \cdots, \text{Room 106}\}\}, \quad A = \{\text{left}, \text{right}\}
\end{equation}
즉, 환경의 상태는 현재 에이전트의 위치 뿐 아니라 배정받은 방이 어디인지도 표현할 수 있어야 합니다.
이를 코드로 보면 아래와 같습니다.

In [ ]:
rooms = list(range(101, 107)) # [101, ..., 106]
state_space = [(i, j) for i in rooms for j in rooms]
action_space = [-1, 1] # left, right

맨 끝 방에서는 반대 방향으로만 이동할 수 있고, 복도 밖으로 이동하려고 해도 벽에 부딪혀 더 움직이지 못합니다. 이를 코드로 구현하면 아래와 같습니다.

In [ ]:
def transition(state, action):
    current_location, my_room = state
    next_location = current_location + action # moves with prob. 1
    next_location = max(next_location, 101) # can't move left
    next_location = min(next_location, 106) # can't move right
    next_state = (next_location, my_room)
    return next_state

다음으로 보상을 정의합니다. 여러분이 배정받은 초록색 방에 도착하면 1의 보상을 받고 환경은 종료됩니다. 그 외의 경우에는 보상이 없습니다.

In [ ]:
def reward_function(state, action):
    next_state = transition(state, action)
    next_location, my_room = next_state
    if next_location == my_room:
        return 1
    return 0

이제 이 환경을 코드로 구현하려면, 크게 두 가지 함수를 정의해야합니다. 첫째, 환경을 생성할 때 에이전트가 처음 관찰할 상태를 제공하는 함수를 만들어야 합니다. 둘째, 에이전트가 환경을 선택했을 때, 다음 상태와 보상을 제공하는 함수를 만들어야 합니다. 아래 코드는 이 두 가지를 각각 $\texttt{reset}$과 $\texttt{step}$ 함수에 구현한 것입니다.

In [ ]:
import numpy as np
import random

# Defines our corridor environment
class CorridorEnv:

    def __init__(self):
        self.start = 101
        self.end = 120
        rooms = list(range(self.start, self.end+1))
        self.state_space = [(i, j) for i in rooms for j in rooms]
        self.action_space = [-1, 1]

    # 위의 코드와 동일!
    def transition(self, state, action):
        current_location, my_room = state
        next_location = current_location + action
        next_location = max(next_location, self.start)
        next_location = min(next_location, self.end)
        next_state = (next_location, my_room)
        return next_state

    # 위의 코드와 동일!
    def reward_function(self, state, action):
        next_state = transition(state, action)
        next_location, my_room = next_state
        if next_location == my_room:
            return 1
        return 0

    def reset(self):
        same_start_and_end = True
        while same_start_and_end:
            state = random.choice(self.state_space)
            same_start_and_end = (state[0] == state[1])
        self.state = state
        return state

    def step(self, action):
        next_state = self.transition(self.state, action)
        reward = self.reward_function(self.state, action)
        done = (next_state[0] == next_state[1])
        self.state = next_state
        return next_state, reward, done

In [ ]:
# Sample code for running the environment
env = CorridorEnv()
state = env.reset()
done = False
print("Initial state:", state)
tr = 0
while not done:
    action = np.random.choice(env.action_space)
    #print("Chose action:", action)
    state, reward, done = env.step(action)
    #print("New state:", state)
    #print("Reward:", reward)
    tr += 1
print(tr)

Initial state: (108, 120)
292


Q. 조금 상황을 바꿔서 복도를 걸어다니는 취한 손님에 대한 환경을 구현해봅시다. 이 환경은 $\texttt{CorridorEnv}$와 거의 동일하지만, 상태 변화에 확률이 추가됩니다. 에이전트는 똑같이 왼쪽 혹은 오른쪽으로 움직일 수 있지만, 이 에이전트는 취해있기 때문에 $20\%$의 확률로 선택한 방향과 반대 방향으로 움직입니다. 이 취한 손님에 대한 환경 $\texttt{DrunkenCorridorEnv}$를 구현해 보세요.

In [ ]:
import numpy as np

# Defines our corridor environment
class DrunkenCorridorEnv:
    def __init__(self):
        self.start = 101
        self.end = 120
        rooms = list(range(self.start, self.end+1))
        self.state_space = [(i, j) for i in rooms for j in rooms]
        self.action_space = [-1, 1]

    def transition(self, state, action):
        if random.randint(0,100)<=20:
          action = action * -1

        current_location, my_room = state
        next_location = current_location + action
        next_location = max(next_location, self.start)
        next_location = min(next_location, self.end)
        next_state = (next_location, my_room)
        return next_state

    def reward_function(self, state, action):
        next_state = transition(state, action)
        next_location, my_room = next_state
        if next_location == my_room:
            return 1
        return 0

    def reset(self):

        same_start_and_end = True
        while same_start_and_end:
            state = random.choice(self.state_space)
            same_start_and_end = (state[0] == state[1])
        self.state = state
        return state

    def step(self, action):
        next_state = self.transition(self.state, action)
        reward = self.reward_function(self.state, action)
        done = (next_state[0] == next_state[1])
        self.state = next_state
        return next_state, reward, done

In [ ]:
env = DrunkenCorridorEnv()
state = env.reset()
done = False
print("Initial state:", state)
tr = 0
while not done:
    action = np.random.choice(env.action_space)
    #print("Chose action:", action)
    state, reward, done = env.step(action)
    #print("New state:", state)
    #print("Reward:", reward)
    tr += 1

print(tr)

Initial state: (118, 119)
1


### 수직선 위의 에이전트
에이전트는 원점의 위치에서 출발하여 보상의 총합을 최대화 하도록 움직이고 싶어합니다. 매 순간 에이전트는 왼쪽 또는 오른쪽으로 이동할 수 있으며, 총 세 번 만 움직일 수 있습니다. 이 때마다 에이전트는 그 위치에 적혀있는 보상을 받습니다. 아래 코드를 통해 에이전트가 놓여있는 환경을 이해해 볼까요?

In [ ]:
class MyEnv(gym.Env):
    def __init__(self):
        self.observation_space = gym.spaces.Discrete(7, start=-3)
        self.action_space = gym.spaces.Discrete(2)
        self.num_steps = 0

    def reset(self):
        state = 0
        return state

    def step(self, action):
        self.num_steps += 1

        if action == 0:
            next_state = state - 1
        else:
            next_state = state + 1

        if next_state > 3:
            next_state = 3
        elif next_state < -3:
            next_state = -3

        reward = {
            -3: 1,
            -2: 1,
            -1: 1,
            0: 0,
            1: -1,
            2: -1,
            3: 10
        }[next_state]

        done = self.num_steps >= 3
        return next_state, reward, done, {}


근시안적인 관점에서 보면, 에이전트가 왼쪽으로 이동해야 당장 더 큰 보상을 받을 수 있습니다. 반대로 오른쪽으로 이동하게 되면 당장은 손해인 것처럼 보입니다. 하지만 총 세 번을 움직일 수 있는 상황에서는 오히려 두 번의 손해를 보고 나서야 비로소 가장 큰 보상을 받을 수 있게 됩니다. 즉, 장기적인 관점에서 에이전트는 보상의 총합을 최대화하기 위해 당장의 손해를 감수해야만 합니다. 이처럼 에이전트는 단순히 현재 상황에서의 최고의 선택을 고르는 것이 아니라 다음 상태까지 모두 고려한 최선의 선택을 내려야합니다.

### 무슨 환경일까요?

In [ ]:
import gymnasium as gym

env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=False)
state, _ = env.reset()
print("Initial state:", state)

done = False
total_reward = 0
while not done:
    action = int(input("Choose action from (0, 1, 2, 3): "))
    print("Chose action:", action)
    state, reward, done, _, _ = env.step(action)
    total_reward += reward
    print("New state:", state)
    print("Reward:", reward)

print("Game over!")
print("Final state:", state)
print("Total reward:", total_reward)
env.close()


Initial state: 0
Choose action from (0, 1, 2, 3): 1
Chose action: 1
New state: 4
Reward: 0.0
Choose action from (0, 1, 2, 3): 1
Chose action: 1
New state: 8
Reward: 0.0
Choose action from (0, 1, 2, 3): 2
Chose action: 2
New state: 9
Reward: 0.0
Choose action from (0, 1, 2, 3): 1
Chose action: 1
New state: 13
Reward: 0.0
Choose action from (0, 1, 2, 3): 1
Chose action: 1
New state: 13
Reward: 0.0
Choose action from (0, 1, 2, 3): 2
Chose action: 2
New state: 14
Reward: 0.0
Choose action from (0, 1, 2, 3): 2
Chose action: 2
New state: 15
Reward: 1.0
Game over!
Final state: 15
Total reward: 1.0


In [ ]:
import gymnasium as gym

env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True) # True로 바뀌면 어떻게 되나요?
state, _ = env.reset()
print("Initial state:", state)

done = False
total_reward = 0
while not done:
    action = int(input("Choose action from (0, 1, 2, 3): "))
    print("Chose action:", action)
    state, reward, done, _, _ = env.step(action)
    total_reward += reward
    print("New state:", state)
    print("Reward:", reward)

print("Game over!")
print("Final state:", state)
print("Total reward:", total_reward)
env.close()

Initial state: 0
Choose action from (0, 1, 2, 3): 2
Chose action: 2
New state: 1
Reward: 0.0
Choose action from (0, 1, 2, 3): 2
Chose action: 2
New state: 5
Reward: 0.0
Game over!
Final state: 5
Total reward: 0.0


# 나만의 강화 학습 환경 만들어보기

In [ ]:
class MyEnv:
    def __init__(self):
        self.state_space = ???
        self.action_space = ???

    def transition(self, state, action):
        next_state = ???
        self.state = next_state
        return next_state

    def reward_function(self, state, action):
        ???
        return reward

    def reset(self):
        ???
        return state

    def step(self, action):
        next_state = self.transition(self.state, action)
        reward = self.reward_function(self.state, action)
        done = ???
        return next_state, reward, done

#환경
한컴타자 해상구조 sos

매 단계마다
상대는 일정 확률로 튜브를 얻음
플레이어와 상대 모두 튜브 1개씩을 잃음


##상태
플레이어와 상대방의 튜브 개수
각자 0이 되면 진다.
동시에 0이되면 비긴다.


##행동
상대의 튜브를 없앤다
-> 상대 튜브 없어질 확률 조정

튜브를 먹어 자신의 튜브를 쌓는다.
-> 튜브가 먹힐 확률 조정


##보상
지면 -1
이기면 1
진행중이거나 비기면 0

